<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Submitting to Quantinuum Emulators

This notebook contains examples for running quantum circuits on Quantinuum's emulators via `pytket`.

An emulator can be used to get an idea of what a quantum device will output for our quantum circuit. This enables circuit debugging and optimization before running on a physical machine. Emulators differ from simulators in that they model the physical and noise model of the device whereas simulators may model noise parameters, but not physical parameters. The Quantinuum emulators run on a physical noise model of the Quantinuum H-Series devices. There are various noise/error parameters modeled. For detailed information on the noise model, see the *Quantinuum System Model H1 Emulator Product Data Sheet* on the user portal.

There are a few options for using the emulator:

1. **Basic Usage:** Use the emulator as provided, which represents both the physical operations in the device as well as the noise. This the most common and simplest way to use the emulator.<br>
2. **Noiseless Emulation:** Use the emulator without the physical noise model applied. The physical device operations are represented, but all errors are set to 0.<br>
3. **Noise Parameters (*advanced option*):** Experiment with the noise parameters in the emulator. There is no guarantee that results achieved changing these parameters will represent outputs from the actual quantum computer represented.<br>
4. **Stabilizer Emulator:** Use of the emulator for circuits involving only Clifford operations.

For more information, see the *Quantinuum System Model H1 Emulator Product Data Sheet*, *Quantinuum Systems User Guide*, and *Quantinuum Application Programming Interface (API)* on the Quantinuum User Portal for detailed information on each of the emulators available and workflow information including job submission, queueing, and the full list of options available.

**Emulator Usage:**<br>
* [Basic Usage](#basic-usage)<br>
* [Noiseless Emulation](#no-noise)<br>
* [Noise Parameters (*advanced*)](#noise)<br>
* [Stabilizer Emulator](#stabilizer)<br>
## Emulator Usage<br>
### Basic Usage <a class="anchor" id="basic-usage"></a>

This section covers usage of the emulator which represents a physical and noise model of the device being used. For example, if using the `H1-1E` target, this emulates the H1-1 quantum computer.

Here the circuit is created via the pytket python library. For details on getting started with `pytket`, see pytket's [Getting Started](https://cqcl.github.io/tket/pytket/api/getting_started.html) page.

In [18]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter

In [19]:
circuit = Circuit(2, name="Bell Test")
circuit.H(0)
circuit.CX(0, 1)
circuit.measure_all()

[H q[0]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [20]:
render_circuit_jupyter(circuit)

Select the emulation device. See the *Quantinuum Systems User Guide* in the *Examples* tab on the *Quantinuum User Portal* for information and target names for each of the emulators available.

In [21]:
from pytket.extensions.quantinuum import QuantinuumBackend

In [22]:
machine = "H1-1E"
backend = QuantinuumBackend(device_name=machine)
#backend.login()

In [51]:
print(machine, "status:", backend.device_state(device_name=machine))

H1-1E status: online


Compile the circuit to the Quantinuum backend with `get_compiled_circuit`. See the `pytket` [User Manual](https://cqcl.github.io/pytket/manual/index.html) for more information on all the options that are available.

In [24]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)

In [25]:
render_circuit_jupyter(compiled_circuit)

Check the circuit HQC cost before running on the emulator.

In [26]:
n_shots = 100
backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker="H1-1SC")

5.66

Run the circuit on the emulator chosen.

In [27]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
print(handle)

('f82e941a336640e9aefbf2c39d4df891', 'null', 2, '[["c", 0], ["c", 1]]')


Check the job status.

In [34]:
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "Bell Test", "submit-date": "2024-03-10T02:33:18.994633", "result-date": "2024-03-10T02:33:32.227780", "queue-position": null, "cost": "5.66", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


Once a job's status returns completed, return results with the `get_result` function.

In [35]:
result = backend.get_result(handle)

It is recommended to save job results as soon as jobs are completed due to the Quantinuum data retention policy.

In [36]:
import json

In [37]:
with open("pytket_emulator_example.json", "w") as file:
    json.dump(result.to_dict(), file)

The result output is just like that of a quantum device. The simulation by default runs with noise.

In [38]:
result = backend.get_result(handle)
print(result.get_distribution())
print(result.get_counts())

{(0, 0): 0.48, (0, 1): 0.01, (1, 0): 0.01, (1, 1): 0.5}
Counter({(1, 1): 50, (0, 0): 48, (0, 1): 1, (1, 0): 1})


### Noiseless Emulation <a class="anchor" id="no-noise"></a>

The Quantinuum emulators may be run with or without the physical device's noise model. The default is the emulator runs with the physical noise model turned on. The physical noise model can be turned off by setting `noisy_simulation=False`.

In [39]:
n_shots = 100
no_error_model_handle = backend.process_circuit(
    compiled_circuit, n_shots=n_shots, noisy_simulation=False
)
print(no_error_model_handle)

('254960af8a5641dea0b545bfa4c1c92d', 'null', 2, '[["c", 0], ["c", 1]]')


In [40]:
no_error_model_status = backend.circuit_status(no_error_model_handle)
print(no_error_model_status)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message='{"name": "Bell Test", "submit-date": "2024-03-10T02:34:01.058405", "result-date": null, "queue-position": null, "cost": "5.66", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [42]:
no_error_model_result = backend.get_result(no_error_model_handle)

In [43]:
with open("pytket_emulator_noiseless_example.json", "w") as file:
    json.dump(result.to_dict(), file)

In [44]:
no_error_model_result = backend.get_result(no_error_model_handle)

In [45]:
print(no_error_model_result.get_distribution())

{(0, 0): 0.49, (1, 1): 0.51}


In [46]:
print(no_error_model_result.get_counts())

Counter({(1, 1): 51, (0, 0): 49})


### Noise Parameters <a class="anchor" id="noise"></a>

The emulator runs with default error parameters that represent a noise environment similar to the physical devices. The `error-params` option can be used to override these error parameters and do finer-grain tweaks of the error model. For detailed information on the noise model, see the *Quantinuum System Model H1 Emulator Product Data Sheet* or the *Quantinuum Application Programming Interface (API)*.

In this section, examples are given for experimenting with the noise and error parameters of the emulators. These are advanced options and not recommended to start with when doing initial experiments. As mentioned above, there is no guarantee that results achieved changing these parameters will represent outputs from the actual quantum computer represented.

**Note**: All the noise parameters are used together any time a simulation is run. If only some of the parameters are specified, the rest of the parameters are used at their default settings. The parameters to override are specified with the `options` parameter.

* [Physical Noise](#physical-noise)<br>
* [Dephasing Noise](#dephasing-noise)<br>
* [Arbitrary Angle Noise Scaling](#arbitrary-angle-noise)<br>
* [Scaling](#scaling)

#### Physical Noise <a class="anchor" id="physical-noise"></a>

See the *Quantinuum System Model H1 Emulator Product Data Sheet* on the Quantinuum user portal for information on these parameters.

In [53]:
machine = "H1-1E"
backend = QuantinuumBackend(device_name=machine)
#backend.login()
render_circuit_jupyter(compiled_circuit)

In [56]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "p1": 4e-5,                         # single qubit fault probability (2.96e-5)
                "p2": 3e-3,                         # two qubit fault probability (1.38e-3)
                "p_meas": 3e-3,                     # bit flip measurement probability (1.32e-3)
                "p_init": 4e-5,                     # initialisation fault probability (3.62e-5)
                "p_crosstalk_meas": 1e-5,           # crosstalk measurement fault probability (8.73e-6)
                "p_crosstalk_init": 3e-5,           # crosstalk initialisation fault probability (5.020e-6)
                "p1_emission_ratio": 0.15,          # ratio of single qubit spontaneous emission to p1 (0.35)
                "p2_emission_ratio": 0.3,           # ratio of single qubit spontaneous in two qubit gates (0.547)
            }
        }
    },
)
result = backend.get_result(handle)

# Physical Noise

The emulators runs with default error parameters that represent a noise environment that closely resembles the respective hardware. These error parameters can be set and used to override the default error parameters and do finer-grain tweaks on the error model


1. **Single-Qubit Fault Probability (p1)** : Probability of a fault occuring during a single qubit gate
2. **Two-Qubit Fault Probability (p2)** :  Probability of a fault occuring during a two-qubit gate
3. **Bit Flip Measurement Probability (p_meas)** : Probability of a bit flip being applied to a measurement. Either a float or a tuple of 2 floats. If it is a single float then the error rate is used to bitflip both $0$ and $1$ measurement results.
4. **Crosstalk Measurement Fault Probability (p_crosstalk_meas)** : Probability of a crosstalk measurement fault occuring per qubit  in the gate zones.
5. **Initialization Fault Probability (p_init)** : Probability of a fault occuring during initialization of a qubit per qubit in the gate zones.
6. **Crosstalke Initialization Fault Probability (p_crosstalk_init)** : Probability of a cross talk fault occuring during initialization of a qubit
7. **Ratio of single-qubit spontaneous emission to p1 (p1_emission_ratio)** : Fraction of p1 that is sponataneous emission for a single qubit instead of asymmetric depolarization noise.
8. **Ratio of single-qubit spontaneous emission in Two Qubit Gates to p2 (p2_emission_ratio)** : Fraction of p2 that is spontaneous emission for a single qubit in a two qubit gate instead of asymmetric depolarizing noise

In [57]:
print(result.get_distribution())

{(0, 0): 0.49, (1, 1): 0.51}


#### Dephasing Noise <a class="anchor" id="dephasing-noise"></a>

See the *Quantinuum System Model H1 Emulator Product Data Sheet* on the user portal for information on these parameters.

In [58]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "quadratic_dephasing_rate": 0.2,
                "linear_dephasing_rate": 0.3,
                "coherent_to_incoherent_factor": 2.0,
                "coherent_dephasing": False,  # False => run the incoherent noise model
                "transport_dephasing": False,  # False => turn off transport dephasing error
                "idle_dephasing": False,  # False => turn off idle dephasing error
            },
        }
    },
)
result = backend.get_result(handle)

## Dephasing Noise
The noise model includes a memory  error for which Z is applied. This is often called "dephasing" or "memory" noise. We potentially model two types of dephasing noise:
1. where the probability of applying Z is quadratically dependent on the duration for which qubits are idling or transporting in the trap
2. where the probability is linearly dependent on the duration

In [59]:
print(result.get_distribution())

{(0, 0): 0.51, (1, 1): 0.49}


#### Arbitrary Angle Noise Scaling <a class="anchor" id="arbitrary-angle-noise"></a>

See the *Quantinuum System Model H1 Emulator Product Data Sheet* on the user portal for information on these parameters.

In [60]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "przz_a": 1.09,
                "przz_b": 0.035,
                "przz_c": 1.09,
                "przz_d": 0.035,
                "przz_power": 1 / 2,
            },
        }
    },
)
result = backend.get_result(handle)

In [61]:
print(result.get_distribution())

{(0, 0): 0.49, (1, 0): 0.01, (1, 1): 0.5}


#### Scaling <a class="anchor" id="scaling"></a>

All the error rates can be scaled linearly using the `scale` parameter. See the *Quantinuum System Model H1 Emulator Product Data Sheet* on the user portal for more information.

In [62]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "scale": 0.1,  # scale error rates linearly by 0.1
            },
        }
    },
)
result = backend.get_result(handle)

In [63]:
print(result.get_distribution())

{(0, 0): 0.49, (1, 1): 0.51}


Other aspects of the noise model can scale specific error rates in the error model, which are modeled here.

In [64]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "error-params": {
                "p1_scale": 0.1,
                "p2_scale": 0.1,
                "meas_scale": 0.1,
                "init_scale": 0.1,
                "memory_scale": 0.1,
                "emission_scale": 0.1,
                "crosstalk_scale": 0.1,
                "leakage_scale": 0.1,
            },
        }
    },
)
result = backend.get_result(handle)

In [65]:
print(result.get_distribution())

{(0, 0): 0.59, (1, 1): 0.41}


### Stabilizer Emulator <a class="anchor" id="stabilizer"></a>

By default, emulations are run using a state-vector emulator, which simulates any quantum operation. However, if the quantum operations are all Clifford gates, it can be faster for complex circuits to use the `stabilizer` emulator. The stabilizer emulator is requested in the setup of the `QuantinuumBackend` with the `simulator` input option. This only applies to Quantinuum emulators.

In [66]:
machine = "H1-1E"

In [67]:
stabilizer_backend = QuantinuumBackend(device_name=machine, simulator="stabilizer")

In [68]:
print(machine, "status:", stabilizer_backend.device_state(device_name=machine))
print("Simulation type:", stabilizer_backend.simulator_type)

H1-1E status: online
Simulation type: stabilizer


In [69]:
n_shots = 100
stabilizer_handle = stabilizer_backend.process_circuit(
    compiled_circuit, n_shots=n_shots
)
print(stabilizer_handle)

('d1c86a7781d14edc9832cb23c37967c5', 'null', 2, '[["c", 0], ["c", 1]]')


In [70]:
stabilizer_status = stabilizer_backend.circuit_status(stabilizer_handle)
print(stabilizer_status)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message='{"name": "Bell Test", "submit-date": "2024-03-10T03:02:56.117907", "result-date": null, "queue-position": null, "cost": null, "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [71]:
stabilizer_result = stabilizer_backend.get_result(stabilizer_handle)
with open("pytket_emulator_stabilizer_example.json", "w") as file:
    json.dump(result.to_dict(), file)

In [72]:
stabilizer_result = stabilizer_backend.get_result(stabilizer_handle)

In [73]:
print(stabilizer_result.get_distribution())

{(0, 0): 0.4, (0, 1): 0.01, (1, 0): 0.01, (1, 1): 0.58}


In [74]:
print(stabilizer_result.get_counts())

Counter({(1, 1): 58, (0, 0): 40, (0, 1): 1, (1, 0): 1})


#### Noiseless Stabilizer

A noiseless stabilizer simulation can be specified via options in the `process_circuit` function with the following options:

- `simulator`: choose to run with a `stabilizer` simulator or `state-vector` (default is `state-vector`)<br>
- `error-model`: whether to run with or without the physical device noise model on or off. The default is `True`, which means the physical noise model is turned on. If set to `False`, the physical noise model is turned off, performing noiseless simulation.

In [75]:
handle = backend.process_circuit(
    compiled_circuit,
    n_shots=100,
    request_options={
        "options": {
            "simulator": "stabilizer",
            "error-model": False,
        }
    },
)
result = backend.get_result(handle)
print(result.get_distribution())

{(0, 0): 0.52, (1, 1): 0.48}


<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>